In [2]:
# 导入必要的库
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse
import os
import math
import itertools
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
import clustree
import shutil

import requests, zipfile

import anndata as ad
import scanpy as sc


# 定义原始文件目录
raw_dir = r"D:\data\GDK_sc_retina\raw_data_normal\GSE228370"
#解压缩
import tarfile

def extract_raw_tar_files(folder_path):
    for filename in os.listdir(folder_path):
        if 'RAW' in filename:
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                try:
                    with tarfile.open(file_path, 'r:*') as tar:
                        tar.extractall(path=folder_path)
                    print(f"成功解压: {filename} 到 {folder_path}")
                except tarfile.TarError as e:
                    print(f"解压失败 {filename}: {e}")

#extract_raw_tar_files(raw_dir)

# 获取所有文件
all_files = os.listdir(raw_dir)

# 提取样本名（去掉后缀）
samples = [f.split('.')[0].rsplit('_', 1)[0] for f in all_files if f.startswith('GSM')]

# 去重样本名
samples = list(set(samples))

# 遍历每个样本
for sample in samples:
    # 找到与当前样本相关的文件
    related_files = [f for f in all_files if sample in f]
    
    # 确保找到的文件数量为 3（barcodes.tsv.gz, features.tsv.gz, matrix.mtx.gz）
    if len(related_files) == 3:
        # 创建目标文件夹
        target_folder = os.path.join(raw_dir, sample)
        os.makedirs(target_folder, exist_ok=True)
        
        # 重命名并移动文件
        for file in related_files:
            # 确定目标文件名
            if "barcodes" in file:
                target_file = os.path.join(target_folder, "barcodes.tsv.gz")
            elif "features" in file:
                target_file = os.path.join(target_folder, "features.tsv.gz")
            elif "matrix" in file:
                target_file = os.path.join(target_folder, "matrix.mtx.gz")
            elif "genes" in file:
                target_file = os.path.join(target_folder, "features.tsv.gz")
            else:
                continue  # 跳过不符合条件的文件
            
            # 检查文件是否存在
            if os.path.exists(os.path.join(raw_dir, file)):
                # 移动并重命名文件
                shutil.move(os.path.join(raw_dir, file), target_file)
            else:
                print(f"File not found: {file}")
        
        print(f"Processed sample: {sample}")
    else:
        print(f"Skipped sample: {sample} (not enough files)")

Skipped sample: GSM7119489 (not enough files)
Skipped sample: GSM7119488 (not enough files)
Skipped sample: GSM7119490 (not enough files)
Skipped sample: GSM7119493_18pcw (not enough files)
Skipped sample: GSM7119492 (not enough files)
Skipped sample: GSM7119496_26pcw_n (not enough files)
Skipped sample: GSM7119494_20pcw (not enough files)
Skipped sample: GSM7119495_23pcw (not enough files)
Skipped sample: GSM7119497_26pcw_o (not enough files)
Skipped sample: GSM7119491 (not enough files)


In [3]:


# 列出目录下的所有样本文件夹
samples = [f for f in os.listdir(raw_dir) if os.path.isdir(os.path.join(raw_dir, f))]

# 存储每个样本的 AnnData 对象
adata_list = []

# 遍历每个样本
for sample in samples:
    print(f"Processing sample: {sample}")
    
    # 构建样本路径
    sample_dir = os.path.join(raw_dir, sample)
    
    # 读取 10X Genomics 数据
    adata = sc.read_10x_mtx(
        sample_dir,  # 样本文件夹路径
        var_names='gene_symbols',  # 使用基因符号作为变量名
        cache=True  # 缓存数据以加快读取速度
    )
    
    # 过滤数据
    sc.pp.filter_cells(adata, min_genes=300)  # 保留至少表达 300 个基因的细胞
    sc.pp.filter_genes(adata, min_cells=5)  # 保留至少在 5 个细胞中表达的基因
    
    # 添加样本信息
    adata.obs['Sample'] = sample
    
    # 将 AnnData 对象添加到列表
    adata_list.append(adata)

# 打印处理结果
print(f"Processed {len(adata_list)} samples.")


Processing sample: GSM7119488_9pcw_scRNA
Processing sample: GSM7119489_10pcw_scRNA
Processing sample: GSM7119490_11pcw_scRNA
Processing sample: GSM7119491_14pcw_scRNA
Processing sample: GSM7119492_15pcw_scRNA
Processing sample: GSM7119493_18pcw_scRNA
Processing sample: GSM7119494_20pcw_scRNA
Processing sample: GSM7119495_23pcw_scRNA
Processing sample: GSM7119496_26pcw_n_scRNA
Processing sample: GSM7119497_26pcw_o_scRNA
Processed 10 samples.


In [4]:
adatas=adata_list

In [5]:
# 合并所有AnnData对象
all_data = adatas[0].concatenate(adatas[1:], batch_key='Sample')

# 可选：若需将batch_key的批次名称设为原Sample值，可添加以下代码
# 提取每个adata的Sample值（假设每个adata的Sample值唯一且恒定）
batch_categories = [adata.obs['Sample'].iloc[0] for adata in adatas]
# 替换批次编号为实际Sample名称
all_data.obs['Sample'] = all_data.obs['Sample'].astype('category').cat.rename_categories(batch_categories)

# 打印合并后的数据
print(all_data)

C:\Users\zhyx\AppData\Local\Temp\ipykernel_25992\2216498342.py:2: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  all_data = adatas[0].concatenate(adatas[1:], batch_key='Sample')


AnnData object with n_obs × n_vars = 93070 × 17495
    obs: 'n_genes', 'Sample'
    var: 'gene_ids', 'feature_types', 'n_cells-0', 'n_cells-1', 'n_cells-2', 'n_cells-3', 'n_cells-4', 'n_cells-5', 'n_cells-6', 'n_cells-7', 'n_cells-8', 'n_cells-9'


In [6]:
all_data.obs["Sample"]

AAACCCAAGCCTGACC-1-0       GSM7119488_9pcw_scRNA
AAACCCAAGCGTCTGC-1-0       GSM7119488_9pcw_scRNA
AAACCCAAGGTCACTT-1-0       GSM7119488_9pcw_scRNA
AAACCCACAAATGATG-1-0       GSM7119488_9pcw_scRNA
AAACCCACAAGCAGGT-1-0       GSM7119488_9pcw_scRNA
                                  ...           
TTTGTTGAGGGCGAAG-1-9    GSM7119497_26pcw_o_scRNA
TTTGTTGAGTAATTGG-1-9    GSM7119497_26pcw_o_scRNA
TTTGTTGCACTAGGCC-1-9    GSM7119497_26pcw_o_scRNA
TTTGTTGTCACCCTTG-1-9    GSM7119497_26pcw_o_scRNA
TTTGTTGTCTGATGGT-1-9    GSM7119497_26pcw_o_scRNA
Name: Sample, Length: 93070, dtype: category
Categories (10, object): ['GSM7119488_9pcw_scRNA', 'GSM7119489_10pcw_scRNA', 'GSM7119490_11pcw_scRNA', 'GSM7119491_14pcw_scRNA', ..., 'GSM7119494_20pcw_scRNA', 'GSM7119495_23pcw_scRNA', 'GSM7119496_26pcw_n_scRNA', 'GSM7119497_26pcw_o_scRNA']

In [ ]:
# 假设 all_data.obs 中有 'Sample' 列存储样本分组信息
print("样本分组示例:\n", all_data.obs['Sample'].head())

# 步骤1：创建样本分组到健康状态的映射字典（根据实际数据修改）
sample_to_health = {
    'GSM5866081': 'glaucoma',
}

# 步骤2：批量添加健康状态列
all_data.obs['health_status'] = (
    all_data.obs['Sample']  # 选择样本分组列
    .map(sample_to_health)  # 应用映射字典
    .astype('category')  # 转换为分类变量节省内存
)

# 步骤3：验证映射完整性
# 检查是否有未映射的样本组
unmapped_samples = all_data.obs[all_data.obs['health_status'].isnull()]['Sample'].unique()
if len(unmapped_samples) > 0:
    print(f"⚠️ 发现未映射样本组: {unmapped_samples}")
else:
    print("✅ 所有样本组均成功映射")

# 步骤4：查看健康状态分布
print("\n健康状态分布统计:")
print(all_data.obs['health_status'].value_counts(dropna=False))


In [ ]:
all_data.var

In [ ]:
all_data.obs

In [ ]:

# 获取 var 列名
var_columns = all_data.var.columns

# 去重并标准化 var 列名
new_var_columns = []
seen = set()
for col in var_columns:
    base_col = col.split('-')[0]  # 去掉后缀
    if base_col not in seen:
        new_var_columns.append(base_col)
        seen.add(base_col)
    else:
        # 如果列名重复，添加唯一后缀
        suffix = 1
        while f"{base_col}_{suffix}" in seen:
            suffix += 1
        new_var_columns.append(f"{base_col}_{suffix}")
        seen.add(f"{base_col}_{suffix}")

# 更新 var 列名
all_data.var.columns = new_var_columns

# 查看更新后的 var 列名
print("Updated var columns:", all_data.var.columns)
# 删除重复的列（例如只保留第一个 gene_ids 和 feature_types）
columns_to_keep = ['gene_ids', 'feature_types', 'n_cells']
all_data.var = all_data.var[columns_to_keep]

# 查看清理后的 var 数据
print("Cleaned var data:", all_data.var.head())
# 查看 obs 数据
print("Cleaned obs data:", all_data.obs.head())

# 查看 var 数据
print("Cleaned var data:", all_data.var.head())

# 查看 AnnData 对象
print(all_data)
all_data.write("adata_combined.h5ad")